In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
os.chdir('/Users/size/DSI/repos/Capstone1/data')

In [6]:
movie_df = pd.read_csv('movies.csv')

In [48]:
def get_dollars(s):
    if s[0] == '$':
        return int(s[s.find(' ') + 1:])
    else:
        return 'Not US'

In [62]:
m1 = movie_df['usa_gross_income'].notna()
m2 = movie_df['budget'].notna()
m_df = movie_df[m1 & m2].copy()
m_df['budget'] = m_df['budget'].apply(get_dollars)
m_df['usa_gross_income'] = m_df['usa_gross_income'].apply(get_dollars)
m_df['worlwide_gross_income'] = m_df['worlwide_gross_income'].apply(get_dollars)
m_df['date_published'] = pd.to_datetime(m_df['date_published'])

In [63]:
m_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8057 entries, 155 to 81059
Data columns (total 22 columns):
imdb_title_id            8057 non-null object
title                    8057 non-null object
original_title           8057 non-null object
year                     8057 non-null int64
date_published           8057 non-null datetime64[ns]
genre                    8057 non-null object
duration                 8057 non-null int64
country                  8056 non-null object
language                 8048 non-null object
director                 8056 non-null object
writer                   8051 non-null object
production_company       8020 non-null object
actors                   8056 non-null object
description              8049 non-null object
avg_vote                 8057 non-null float64
votes                    8057 non-null int64
budget                   8057 non-null object
usa_gross_income         8057 non-null int64
worlwide_gross_income    8057 non-null int64
metascore   

In [85]:
g = [len(x.split(', ')) for x in m_df['genre']]
min(g)

1

In [80]:
for x in m_df['genre'][0:1]:
    print(x.split(', '))

['Fantasy', 'Horror', 'Mystery']


In [54]:
m_df['budget'][:10]

155       18000
232      800000
488      Not US
1005    1500000
2336    1500000
2687    1499000
3100    3977000
3180    1900000
3219     Not US
3277    2777000
Name: budget, dtype: object

In [41]:
y = movie_df['usa_gross_income'][0]

In [45]:
y

nan